In [54]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

module_path = (Path().resolve().parent/ "Modules")
sys.path.append(str(module_path))

pd.set_option("display.max_columns", None)

import my_modules, model_tuner, features # 自作モジュール

In [55]:
df = pd.read_csv("../Data/Time_Series_Odds/JT06201101.csv", encoding="shift-jis")
df.head()

,レースID,区分,月日時分,頭数,単勝票数,1番,2番,3番,4番,5番,6番,7番,8番,9番,10番,11番,12番,13番,14番,15番,16番
0,2020010506010101,1,1041831,16,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020010506010101,1,1041836,16,35,9.3,0.0,0.0,0.0,0.0,0.0,28.0,0.0,1.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020010506010101,1,1041841,16,110,6.2,0.0,0.0,0.0,0.0,0.0,1.4,0.0,2.7,0.0,0.0,44.0,0.0,0.0,0.0,0.0
3,2020010506010101,1,1041846,16,304,11.5,24.3,0.0,121.6,0.0,0.0,1.2,48.6,7.1,48.6,0.0,14.3,243.2,0.0,48.6,48.6
4,2020010506010101,1,1041851,16,346,13.1,27.6,0.0,92.2,39.5,0.0,1.3,46.1,4.2,46.1,0.0,16.2,276.8,0.0,55.3,55.3


- オッズの下2行は確定オッズ
- 区分1, 3, 4で分かれている
  - 1 ... 通常オッズ？
  - 3 ... 締め切り時刻のオッズ？
  - 4 ... 確定オッズ？

In [56]:
df.columns = df.columns.str.replace(" ","")
df.columns

Index(['レースID', '区分', '月日時分', '頭数', '単勝票数', '1番', '2番', '3番', '4番', '5番', '6番',
       '7番', '8番', '9番', '10番', '11番', '12番', '13番', '14番', '15番', '16番'],
      dtype='object')

In [57]:
def convert_to_chart(df):
    df = df.copy()
    df.columns = df.columns.str.replace(" ","")
    df_normal_odds = df[df["区分"] == 1]
    df_filtered = df_normal_odds.filter(regex=r'^\d+番$', axis=1)
    df_filtered.columns = df_filtered.columns.str.replace("番", "", regex=False)
    df_filtered = df_filtered.T
    df_filtered = df_filtered[df_filtered.columns[-20:]] # 最後の20列だけ使う
    df_filtered.columns = [f"win_odds_{i+1}" for i in range(df_filtered.shape[1])]
    df_filtered = df_filtered.reset_index(drop=False).rename(columns={"index": "horse_num"})
    df_filtered["race_id"] = df["レースID"].unique()[0] * 100 + df_filtered["horse_num"].astype(int)

    return df_filtered

df_chart = convert_to_chart(df)
df_chart.head()

,horse_num,win_odds_1,win_odds_2,win_odds_3,win_odds_4,win_odds_5,win_odds_6,win_odds_7,win_odds_8,win_odds_9,win_odds_10,win_odds_11,win_odds_12,win_odds_13,win_odds_14,win_odds_15,win_odds_16,win_odds_17,win_odds_18,win_odds_19,win_odds_20,race_id
0,1,15.2,15.3,14.9,15.1,15.2,15.4,15.3,15.2,15.5,15.4,14.8,15.1,15.0,14.5,15.3,15.1,16.1,15.9,16.1,16.2,202001050601010101
1,2,20.1,20.7,20.9,21.1,21.5,21.7,22.0,22.2,22.6,22.6,23.2,23.8,24.4,24.5,26.1,26.8,28.3,29.0,31.1,34.3,202001050601010102
2,3,163.5,160.3,157.6,139.1,142.6,135.9,138.8,139.2,140.0,135.5,138.9,144.8,149.0,151.5,158.9,157.2,151.1,152.6,158.9,172.8,202001050601010103
3,4,21.0,21.6,21.6,21.4,21.6,21.0,20.6,19.9,19.7,19.5,19.1,19.7,19.7,19.1,19.3,18.7,19.1,17.1,15.7,15.0,202001050601010104
4,5,50.2,50.7,50.7,50.1,50.1,50.7,51.3,51.8,51.6,50.9,51.2,53.6,55.0,54.3,58.6,57.7,57.9,59.8,65.8,76.2,202001050601010105


In [ ]:
import os
import glob # ファイルパスのパターンマッチングに便利

# --- メイン処理 ---
# 全てのCSVファイルを処理し、結合して一つのCSVファイルに出力する関数
def process_and_combine_csv_files(folder_path, output_file_name="combined_output.csv"):
    """
    指定されたフォルダ内のすべてのCSVファイルを処理し、結合して一つのCSVファイルに出力します。

    Args:
        folder_path (str): CSVファイルが格納されているフォルダのパス。
        output_file_name (str): 結合されたCSVファイルの出力名。
    """
    
    all_processed_dfs = [] # 処理後のDataFrameを格納するリスト

    # フォルダ内のすべてのCSVファイルのパスを取得
    # os.path.joinを使ってプラットフォームに依存しないパスを作成します
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

    if not csv_files:
        print(f"指定されたフォルダ '{folder_path}' にCSVファイルが見つかりませんでした。")
        return

    print(f"--- 処理を開始します: {len(csv_files)}個のCSVファイルを検出 ---")

    for i, file_path in enumerate(csv_files):
        print(f"ファイル {i+1}/{len(csv_files)}: '{os.path.basename(file_path)}' を読み込み中...")
        try:
            # CSVファイルを読み込み
            df = pd.read_csv(file_path, encoding="shift-jis")
            
            # 定義された関数でDataFrameを処理
            processed_df = convert_to_chart(df)
            
            # 処理後のDataFrameをリストに追加
            all_processed_dfs.append(processed_df)
            
        except Exception as e:
            print(f"エラー: ファイル '{os.path.basename(file_path)}' の処理中にエラーが発生しました: {e}")
            continue # エラーが発生しても次のファイルへ進む

    if not all_processed_dfs:
        print("処理に成功したファイルがありませんでした。結合はスキップされます。")
        return

    # すべての処理済みDataFrameを結合
    # ignore_index=True で新しい連続したインデックスを生成します
    combined_df = pd.concat(all_processed_dfs, ignore_index=True, axis=0)

    # 結合したDataFrameを新しいCSVファイルとして保存
    output_path = os.path.join(os.getcwd(), output_file_name) # 現在のディレクトリに出力
    combined_df.to_csv(output_path, index=False)

    print(f"\n--- 処理が完了しました ---")
    print(f"結合されたファイルは '{output_path}' に保存されました。")
    print(f"最終的なDataFrameの形状: {combined_df.shape}")

In [62]:
process_and_combine_csv_files("../Data/Time_Series_Odds_Test/", output_file_name="Time_Series_Odds_win_odds.csv")

--- 処理を開始します: 18173個のCSVファイルを検出 ---
ファイル 1/18173: 'JT01201101.CSV' を読み込み中...
ファイル 2/18173: 'JT01201102.CSV' を読み込み中...
ファイル 3/18173: 'JT01201103.CSV' を読み込み中...
ファイル 4/18173: 'JT01201104.CSV' を読み込み中...
ファイル 5/18173: 'JT01201105.CSV' を読み込み中...
ファイル 6/18173: 'JT01201106.CSV' を読み込み中...
ファイル 7/18173: 'JT01201107.CSV' を読み込み中...
ファイル 8/18173: 'JT01201108.CSV' を読み込み中...
ファイル 9/18173: 'JT01201109.CSV' を読み込み中...
ファイル 10/18173: 'JT01201110.CSV' を読み込み中...
ファイル 11/18173: 'JT01201111.CSV' を読み込み中...
ファイル 12/18173: 'JT01201112.CSV' を読み込み中...
ファイル 13/18173: 'JT01201201.CSV' を読み込み中...
ファイル 14/18173: 'JT01201202.CSV' を読み込み中...
ファイル 15/18173: 'JT01201203.CSV' を読み込み中...
ファイル 16/18173: 'JT01201204.CSV' を読み込み中...
ファイル 17/18173: 'JT01201205.CSV' を読み込み中...
ファイル 18/18173: 'JT01201206.CSV' を読み込み中...
ファイル 19/18173: 'JT01201207.CSV' を読み込み中...
ファイル 20/18173: 'JT01201208.CSV' を読み込み中...
ファイル 21/18173: 'JT01201209.CSV' を読み込み中...
ファイル 22/18173: 'JT01201210.CSV' を読み込み中...
ファイル 23/18173: 'JT01201211.CSV' を読み込み中...
ファイル 24

In [64]:
odds_df = pd.read_csv("Time_Series_Odds_win_odds.csv", encoding="shift-jis")
odds_df.head()

,horse_num,win_odds_1,win_odds_2,win_odds_3,win_odds_4,win_odds_5,win_odds_6,win_odds_7,win_odds_8,win_odds_9,win_odds_10,win_odds_11,win_odds_12,win_odds_13,win_odds_14,win_odds_15,win_odds_16,win_odds_17,win_odds_18,win_odds_19,win_odds_20,race_id
0,1,16.1,16.1,16.3,16.3,16.2,16.2,15.8,15.8,15.8,15.9,15.9,15.6,16.0,15.9,17.3,17.0,16.7,17.9,19.6,20.9,202007250101010101
1,2,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8,1.8,1.8,1.8,1.8,1.8,202007250101010102
2,3,2.2,2.2,2.2,2.3,2.3,2.3,2.3,2.2,2.2,2.3,2.3,2.3,2.3,2.2,2.1,2.1,2.1,2.0,2.0,1.9,202007250101010103
3,4,40.2,39.9,40.8,38.7,38.6,37.6,37.5,37.9,37.7,37.7,38.0,38.3,39.6,38.0,40.6,42.0,42.9,46.4,54.6,52.8,202007250101010104
4,5,40.2,40.3,41.3,41.9,41.7,40.6,40.6,41.2,41.5,42.0,41.5,42.2,43.7,43.7,46.9,47.6,48.9,51.5,60.8,61.2,202007250101010105


In [65]:
odds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252819 entries, 0 to 252818
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   horse_num    252819 non-null  int64  
 1   win_odds_1   252819 non-null  float64
 2   win_odds_2   252819 non-null  float64
 3   win_odds_3   252819 non-null  float64
 4   win_odds_4   252819 non-null  float64
 5   win_odds_5   252819 non-null  float64
 6   win_odds_6   252819 non-null  float64
 7   win_odds_7   252819 non-null  float64
 8   win_odds_8   252819 non-null  float64
 9   win_odds_9   252819 non-null  float64
 10  win_odds_10  252819 non-null  float64
 11  win_odds_11  252774 non-null  float64
 12  win_odds_12  252726 non-null  float64
 13  win_odds_13  252679 non-null  float64
 14  win_odds_14  252622 non-null  float64
 15  win_odds_15  252558 non-null  float64
 16  win_odds_16  252526 non-null  float64
 17  win_odds_17  252485 non-null  float64
 18  win_odds_18  252452 non-